In [2]:
#%pip install plotly

import pandas as pd
import json
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
with open('data/top_500songs_detailed.json', encoding='utf-8') as f:
    data = json.load(f)

df = pd.json_normalize(
    data,
    record_path='artists',
    meta=[
        'track_id', 'track_name', 'popularity', 
        'duration_ms', 'explicit', 'release_date', 
        'album_name', 'spotify_url'
    ],
    meta_prefix='track_'
)
df.head()

,artist_id,artist_name,genres,popularity,followers,track_track_id,track_track_name,track_popularity,track_duration_ms,track_explicit,track_release_date,track_album_name,track_spotify_url
0,7bXgB6jMjp9ATFy66eO08Z,Chris Brown,[r&b],89,26556058,32SG0ApCqF9qnleIl4mK67,Luckiest Man,37,150720,False,2022-06-23,Breezy,https://open.spotify.com/track/32SG0ApCqF9qnle...
1,10HyhQTjkmDpWBpaUgO2GS,Flowerboy,"[bedroom pop, indie r&b]",26,3280,53oZXP24LECwD6YBybecvG,Plant!,25,131500,True,2021-09-29,Plant!,https://open.spotify.com/track/53oZXP24LECwD6Y...
2,699OTQXzgjhIYAHMy9RyPD,Playboi Carti,[rage rap],92,14267846,2cZOYofOX4d6g0OXxkaIjA,ALL RED,75,148053,True,2024-09-13,ALL RED,https://open.spotify.com/track/2cZOYofOX4d6g0O...
3,0k7xoR8j1LbA4VXbKCH5dL,Kieran Alleyne,[],37,9346,04f7fQILKDgLE8aHbF73VO,Runnin Low,53,231273,True,2014-10-17,Runnin Low,https://open.spotify.com/track/04f7fQILKDgLE8a...
4,7mcE0EmrnPmyAGU3UBJrFg,LOVELI LORI,[],66,147752,0nBZciKoSZ9bHnV6DmMdUd,get u off my mind,52,143810,False,2024-02-17,get u off my mind,https://open.spotify.com/track/0nBZciKoSZ9bHnV...


can do genres, popularity, track popularity, and duration, time released.

In [ ]:
df1 = df[df['genres'].apply(lambda x: len(x) == 0)]
df1.to_csv('data/top_500songs_no_genres.csv', index=False)

In [35]:
# import spotipy 
# from spotipy.oauth2 import SpotifyOAuth
# from dotenv import load_dotenv
# import os
# import pandas as pd

# # Load environment variables
# load_dotenv()

# CLIENT_ID = os.getenv('SPOTIPY_CLIENT_ID')
# CLIENT_SECRET = os.getenv('SPOTIPY_CLIENT_SECRET')
# REDIRECT_URI = os.getenv('SPOTIPY_REDIRECT_URI', 'http://localhost:8000/callback')
# SCOPE = 'user-top-read'

# # Initialize Spotify API client
# sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
#     client_id=CLIENT_ID,
#     client_secret=CLIENT_SECRET,
#     redirect_uri=REDIRECT_URI,
#     scope=SCOPE
# ))

# # Read the CSV with missing genres
# #df1 = pd.read_csv("data/top_500songs_no_genres.csv").copy()

# # Function to get genres by searching track -> extracting artist -> querying artist
# def get_genres(track_name, artists):
#     try:
#         query = f"track:{track_name} artist:{' '.join(artists)}"
#         results = sp.search(q=query, type='track', limit=1)
#         if results['tracks']['items']:
#             artist_id = results['tracks']['items'][0]['artists'][0]['id']
#             artist_info = sp.artist(artist_id)
#             return artist_info.get('genres', [])
#         else:
#             return []
#     except Exception as e:
#         print(f"Error for {track_name}: {e}")
#         return []

# # Apply the function row-wise
# df['genres'] = df.apply(lambda row: get_genres(row['track_track_name'], [row['artist_name']]), axis=1)

# # Save updated DataFrame
# df.to_csv("data/top_500songs_with_fixed_genres.csv", index=False)

# ## update the df1 with new genres from top_500songs_with_fixed_genres.csv
try:
	df = pd.read_csv("data/top_500songs_with_fixed_genres.csv", encoding='utf-8').copy()
except UnicodeDecodeError:
	df = pd.read_csv("data/top_500songs_with_fixed_genres.csv", encoding='latin1').copy()

import ast

def safe_len_genres(x):
	if pd.isna(x) or x.strip() == "":
		return 0
	try:
		genres_list = ast.literal_eval(x)
		return len(genres_list)
	except Exception:
		return 0

total_empty_genres = (df['genres'].apply(safe_len_genres) == 0).sum()
print(f"Total songs with empty genres: {total_empty_genres}")

Total songs with empty genres: 149


In [37]:
df.columns

Index(['artist_id', 'artist_name', 'genres', 'popularity', 'followers',
       'track_track_id', 'track_track_name', 'track_popularity',
       'track_duration_ms', 'track_explicit', 'track_release_date',
       'track_album_name', 'track_spotify_url', 'num_genres'],
      dtype='object')

In [36]:
df.shape

(751, 14)

In [24]:
print("df shape: ",df.shape)
print("df1 shape: ",df1.shape)

df shape:  (751, 13)
df1 shape:  (269, 13)


In [38]:
df_exploded = df.explode('genres')
df_exploded.shape

(751, 14)

In [ ]:
#%pip install scikit-learn

from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
genre_matrix = pd.DataFrame(mlb.fit_transform(df['genres']), columns=mlb.classes_)
df = pd.concat([df, genre_matrix], axis=1)


  Using cached scikit_learn-1.6.1-cp313-cp313-win_amd64.whl.metadata (15 kB)
  Using cached scipy-1.15.3-cp313-cp313-win_amd64.whl.metadata (60 kB)
  Using cached joblib-1.5.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.6.1-cp313-cp313-win_amd64.whl (11.1 MB)
Using cached joblib-1.5.1-py3-none-any.whl (307 kB)
Using cached scipy-1.15.3-cp313-cp313-win_amd64.whl (41.0 MB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)

   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- -----------------

In [23]:
df_exploded['genres'] = df_exploded['genres'].replace('', np.nan)
df_exploded = df_exploded.dropna(subset=['genres'])

In [24]:
genre_counts = df_exploded['genres'].value_counts().reset_index()
genre_counts.columns = ['genre', 'count']

In [36]:
# top_genres = genre_counts.head(15)
# sns.barplot(data=top_genres, x='count', y='genre', palette='viridis')
# plt.title('Top Genres in My Spotify Top 500')
# plt.xlabel('Count')
# plt.ylabel('Genre')
# plt.tight_layout()
# plt.show()

top_genres = df_exploded['genres'].value_counts().head(20).reset_index()
top_genres.columns = ['genre', 'count']

fig = px.bar(
    top_genres,
    x='count',
    y='genre',
    orientation='h',
    color='count',
    color_continuous_scale='plasma',
    title='Top 20 Genres in My Spotify Top 500',
    labels={'count': 'Number of Tracks', 'genre': 'Genre'},
    template= 'plotly_dark'
)

fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.write_html("plot.html")

In [ ]:
# 120,000 ms = 2 minutes 
# 180,000 ms = 3 minutes
# 240,000 ms = 4 minutes

In [19]:
from scipy.stats import entropy

def genre_entropy(genres):
    counts = pd.Series(genres).value_counts()
    return entropy(counts)

artist_entropy = df_exploded.groupby('artist_name')['genres'].apply(genre_entropy)
artist_entropy

artist_name
125 Rue Montmartre     1.609438
21 Savage              0.000000
5 Seconds of Summer    0.000000
6LACK                  0.000000
A$AP Rocky             0.000000
                         ...   
wickbabi               0.000000
wolf.cryman            1.386294
ඞ                      0.000000
ヨルシカ                   1.386294
小畑貴裕                   0.000000
Name: genres, Length: 513, dtype: float64